In [171]:
import pandas as pd
import re
import glob
from sqlalchemy import create_engine



In [172]:
folder_path = 'I:/Expense tracking/Apple Pay'
# folder_path='D:/Personal Projects/Expense tracking/Apple Pay'
file_pattern = folder_path + '/*.csv'  # Adjust the pattern if using other formats, e.g., '*.xlsx'

# Use glob to find all files matching the pattern
file_list = glob.glob(file_pattern)

# Read all files and concatenate them into a single DataFrame
df_apple_pay = pd.concat((pd.read_csv(file) for file in file_list), ignore_index=True)

In [173]:
df_apple_pay.head()

,Transaction Date,Clearing Date,Description,Merchant,Category,Type,Amount (USD),Purchased By
0,04/29/2024,04/30/2024,GOODYEAR ASC #4181 8012 COIT ROAD PLANO 75024 ...,Goodyear,Other,Purchase,117.49,Sandeep Rani Pradeep Kumar
1,04/30/2024,04/30/2024,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,Ach Deposit Internet Transfer From Account End...,Payment,Payment,-165.62,Sandeep Rani Pradeep Kumar
2,04/23/2024,04/24/2024,DUNKIN #346310 7317 PARKWOOD BOULEVARD PLANO 7...,Dunkin',Restaurants,Purchase,2.81,Sandeep Rani Pradeep Kumar
3,04/22/2024,04/23/2024,FEDEX OFFIC798000079898290 STATE HWY 121 FRISC...,FedEx,Other,Purchase,6.54,Sandeep Rani Pradeep Kumar
4,04/19/2024,04/20/2024,WDW TICKETS 1375 E BUENA VISTA DR LAKE BUENA V...,WDW Tickets,Entertainment,Purchase,381.28,Sandeep Rani Pradeep Kumar


In [174]:
# df_apple_pay['Transaction Date'] = pd.to_datetime(df_apple_pay['Transaction Date'])
# df_apple_pay['Transaction Date']=df_apple_pay['Transaction Date'].dt.strftime('%m/%d/%y')
# # df_apple_pay['Transaction Date']=pd.to_datetime(df_apple_pay['Transaction Date'], format='%m/%d/%y')

In [175]:
df_apple_pay=df_apple_pay[['Transaction Date','Description','Category','Amount (USD)','Merchant']]
df_apple_pay['Institution']='Apple Pay Credit Card'
df_apple_pay['Is Hidden']='No'
df_apple_pay['Is Pending']='No'
df_apple_pay['Account']='APPLE PAY'


In [176]:
df_apple_pay.rename(columns={"Merchant":"Standardized Description","Amount (USD)":"Amount"},inplace=True)
dtype_dict = {
    'Transaction Date':'datetime64[ns]',
    'Description': 'string',
    'Institution': 'string',
    'Account': 'string',
    'Category': 'string',
    'Is Hidden': 'string',
    'Is Pending': 'string',
    'Amount': 'float64',
    'Standardized Description': 'string'
}
df_apple_pay = df_apple_pay.astype(dtype_dict)
# df_apple_pay['Transaction Date']=df_apple_pay['Transaction Date'].dt.strftime('%m/%d/%y')
df_apple_pay=df_apple_pay[['Transaction Date','Description','Institution','Account','Category','Is Hidden','Is Pending','Amount','Standardized Description']]


In [177]:
df_apple_pay.head()

,Transaction Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,Standardized Description
0,2024-04-29,GOODYEAR ASC #4181 8012 COIT ROAD PLANO 75024 ...,Apple Pay Credit Card,APPLE PAY,Other,No,No,117.49,Goodyear
1,2024-04-30,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,Apple Pay Credit Card,APPLE PAY,Payment,No,No,-165.62,Ach Deposit Internet Transfer From Account End...
2,2024-04-23,DUNKIN #346310 7317 PARKWOOD BOULEVARD PLANO 7...,Apple Pay Credit Card,APPLE PAY,Restaurants,No,No,2.81,Dunkin'
3,2024-04-22,FEDEX OFFIC798000079898290 STATE HWY 121 FRISC...,Apple Pay Credit Card,APPLE PAY,Other,No,No,6.54,FedEx
4,2024-04-19,WDW TICKETS 1375 E BUENA VISTA DR LAKE BUENA V...,Apple Pay Credit Card,APPLE PAY,Entertainment,No,No,381.28,WDW Tickets


In [178]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [179]:
def convert_accounting_to_decimal(amount):
    # Remove parentheses and commas
    amount = amount.replace('$', '').replace('(', '-').replace(')', '').replace(',', '')
    # Convert to float
    return float(amount)

In [180]:

file_names = ['Transactions 2022_06-2023_06.csv', 'Transactions 2023_06-2024_06.csv']
folder_path = 'I:/Expense tracking'
# folder_path='D:/Personal Projects/Expense tracking'
file_pattern = folder_path + '/*.csv'  # Adjust the pattern if using other formats, e.g., '*.xlsx'

# Use glob to find all files matching the pattern
file_list = glob.glob(file_pattern)
print(file_list)

# Load and concatenate the files into a single DataFrame
df = pd.concat([pd.read_csv(file) for file in file_names], ignore_index=True)


['I:/Expense tracking\\Transactions 2022_06-2023_06.csv', 'I:/Expense tracking\\Transactions 2023_06-2024_06.csv', 'I:/Expense tracking\\Transactions Full.csv']


In [181]:
df['Amount']=df['Amount'].apply(convert_accounting_to_decimal)

In [182]:
df['Standardized Description']=''
df.rename(columns={"Date":"Transaction Date"},inplace=True)


In [183]:

dtype_dict = {
    'Transaction Date':'datetime64[ns]',
    'Description': 'string',
    'Institution': 'string',
    'Account': 'string',
    'Category': 'string',
    'Is Hidden': 'string',
    'Is Pending': 'string',
    'Amount': 'float64',
    'Standardized Description': 'string'
}
df = df.astype(dtype_dict)
# len(df)

In [184]:
df['Account'].unique()

<StringArray>
[               'DISCOVER',                    'AMEX',
          'CHASE CHECKING',           'CHASE FREEDOM',
               'ROBINHOOD',           'CHASE SAVINGS',
           'GOLDMAN SACHS', 'CHARLES SCHWAB CHECKING',
    'WELLS FARGO CHECKING']
Length: 9, dtype: string

In [185]:
len(df)

2417

In [186]:
df_full=pd.concat([df,df_apple_pay],ignore_index=True)

In [187]:
len(df_full)

2485

In [188]:
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
df_full[df_full['Account']=='APPLE PAY']['Category'].unique()
df_full[df_full['Category']=='Medical']

,Transaction Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,Standardized Description
2220,2023-08-08,NAYAX COPY SERVICES,Chase - via Direct Web API,CHASE FREEDOM,Medical,No,No,-3.00,
2221,2023-08-08,NAYAX COPY SERVICES,Chase - via Direct Web API,CHASE FREEDOM,Medical,No,No,-0.45,
2435,2024-02-22,THE JOINT PRESTON ROA8240 PRESTON ROAD STE 165...,Apple Pay Credit Card,APPLE PAY,Medical,No,No,118.00,The Joint
2452,2024-06-11,THE JOINT PRESTON ROA8240 PRESTON ROAD STE 165...,Apple Pay Credit Card,APPLE PAY,Medical,No,No,89.00,The Joint
2461,2024-03-22,THE JOINT PRESTON ROA8240 PRESTON ROAD STE 165...,Apple Pay Credit Card,APPLE PAY,Medical,No,No,89.00,The Joint


In [189]:
df_full.head()

,Transaction Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,Standardized Description
0,2023-06-13,AMZN MKTP US AMZN.COM BILLWA,Discover Credit Card - via Direct Web Access,DISCOVER,Shopping,No,No,-16.63,
1,2023-06-13,FRISCO TX,Discover Credit Card - via Direct Web Access,DISCOVER,Gas & Fuel,No,No,-30.00,
2,2023-06-13,AMZN MKTP US AMZN.COM BILLWA,Discover Credit Card - via Direct Web Access,DISCOVER,Shopping,No,No,-23.99,
3,2023-06-12,APLPAY STARBUCKS WA,American Express - Amex Account - via Direct W...,AMEX,Coffee,No,No,-10.00,
4,2023-06-12,ZELLE PAYMENT TO BOOKSHELF,Chase - via Direct Web API,CHASE CHECKING,Online Payment,No,No,-10.00,


In [190]:
df_full['Category'].unique()

<StringArray>
[                    'Shopping',                   'Gas & Fuel',
                       'Coffee',               'Online Payment',
                       'Income',           'Restaurants/Dining',
      'Fast Food & Convenience',                    'Groceries',
                   'Rental Car',      'Phone, Internet & Cable',
                          'Gym',                        'Tolls',
                    'Rideshare',                'Subscriptions',
                 'Auto Service',           'Investment Savings',
                         'Rent',         'Investment Transfers',
          'Shipping & Handling',                      'Savings',
                    'Transfers',              'Interest Income',
               'Fees & Charges',             'Auto & Transport',
          'Credit Card Payment',                        'Hotel',
       'Furniture & Home Decor',           'Household Services',
                     'Cash/ATM',                     'Bank Fee',
           

In [191]:
patterns_map = {
    'COFFEE': [r'STARBUCKS', r'DUNKIN',r'PANERA'],
    'SHOPPING':[r'WALMART',r'TARGET'],
    'RENT':[r'BELL WEB',r'SUMMIT',r'PALMS ASSOCIATES'],
    'ENERGY, GAS & ELECTRIC':[r'TXU'],
    'RESTAURANTS/DINING':[r'HELLO INDIA',r'HELLO! INDIA'],
    'SUBSCRIPTIONS':[r'ENVATO',r'STRATA',r'VIDIQ',r'YOUTUBE',r'HUNTER.IO'],
    'PAYMENT':[r'USCIS',r'ZELLE PAYMENT TO'],
    'INCOME':[r'ZELLE PAYMENT FROM'],
    'ENTERTAINMENT':[r'SIXFLAGS'],
    'GYM':[r'CRUNCH']
}

# Function to map the new category
def map_category(row):
    description_upper = row['Description'].upper()  # Convert to uppercase for case-insensitive matching
    for category, patterns in patterns_map.items():
        if any(re.search(pattern, description_upper) for pattern in patterns):
            return category
    return row['Category'].upper()
    
df_full['Category'] = df_full.apply(map_category, axis=1)



In [192]:
master_category_dict={
'SHOPPING/GROCERIES':['Groceries','Grocery','Shopping','Furniture & Home Decor','Merchandise/Misc','Gifts','Clothing','Home Improvement/Maintenance','Books','Electronics & Software'],
'ENTERTAINMENT':['Entertainment','Air Travel','Hotel','Rental Car','Travel & Vacation'],
'FOOD AND DINING':['Restaurants','Restaurants/Dining','Fast Food & Convenience',	'Coffee','Food','Alcohol & Bars'],
'SUBSCRIPTIONS':['Subscriptions','Education'],
'HEALTH':['Gym'],
'BILLS AND UTILITIES':['Phone, Internet & Cable','Gas & Fuel','Gas','Insurance','Auto Service','Energy, Gas & Electric','Auto & Transport',
                       	'Water'	,'Bills & Utilities','Rent','Tolls','Rideshare','Parking','Pharmacy','Hair & Nails','Medical','Household Services'],
'MISCELLANEOUS':['Transfers','Credit Card Payment','Savings','Investment Savings','Investment Transfers','Online Payment','Bank Fee'
                 	,'Fees & Charges','Shipping & Handling','Taxes','Cash/ATM','Charity','Payment','Other'],
'INCOME':['Interest Income','Paycheck/Salary','Tax Refund','Income']
    
}

def map_category(row):
    for category, patterns in master_category_dict.items():
        if any(pattern.lower() in row['Category'].lower() for pattern in patterns):
            return category
    return 'OTHER'
    
df_full['Master Category'] = df_full.apply(map_category, axis=1)


In [193]:
df_full['Master Category'].unique()

array(['SHOPPING/GROCERIES', 'BILLS AND UTILITIES', 'FOOD AND DINING',
       'MISCELLANEOUS', 'INCOME', 'HEALTH', 'SUBSCRIPTIONS',
       'ENTERTAINMENT'], dtype=object)

In [194]:
df_full.head()

,Transaction Date,Description,Institution,Account,Category,Is Hidden,Is Pending,Amount,Standardized Description,Master Category
0,2023-06-13,AMZN MKTP US AMZN.COM BILLWA,Discover Credit Card - via Direct Web Access,DISCOVER,SHOPPING,No,No,-16.63,,SHOPPING/GROCERIES
1,2023-06-13,FRISCO TX,Discover Credit Card - via Direct Web Access,DISCOVER,GAS & FUEL,No,No,-30.00,,BILLS AND UTILITIES
2,2023-06-13,AMZN MKTP US AMZN.COM BILLWA,Discover Credit Card - via Direct Web Access,DISCOVER,SHOPPING,No,No,-23.99,,SHOPPING/GROCERIES
3,2023-06-12,APLPAY STARBUCKS WA,American Express - Amex Account - via Direct W...,AMEX,COFFEE,No,No,-10.00,,FOOD AND DINING
4,2023-06-12,ZELLE PAYMENT TO BOOKSHELF,Chase - via Direct Web API,CHASE CHECKING,PAYMENT,No,No,-10.00,,MISCELLANEOUS


In [195]:
desc_map = {
    'STARBUCKS': [r'STARBUCKS'],
    'DUNKIN': [r'DUNKIN'],
    '7-ELEVEN':[r'7-ELEVEN'],
    'WALMART':[r'WALMART',r'WM',r'WAL-MART'],
    'TARGET':[r'TARGET'],
    'HOME DEPOT':[r'HOME DEPOT'],
    'AMAZON':[r'AMAZON',r'AMZN'],
    'COSTCO':[r'COSTCO'],
    'HELLO INDIA': [r'HELLO INDIA',r'HELLO! INDIA'],
    'PANERA':[r'PANERA'],
    'TELADOC':[r'TELADOC'],
    'SIGNIFY':[r'SIGNIFY'],
    'CMA CGM':[r'CMA CGM']
}

def map_description(row):
    description_upper = row['Description'].upper()  # Convert to uppercase for case-insensitive matching
    for category, patterns in desc_map.items():
        if any(re.search(pattern, description_upper) for pattern in patterns):
            return category
    return row['Description'].upper()
    

# Apply the function to create a new column 'Mapped Description'
df_full['Standardized Description'] = df_full.apply(map_description,axis=1)


Filter out 7 ELEVEN as it needs cleaning

In [196]:

filtered_df = df_full[~(df_full['Description'].str.match('7-ELEVEN', case=False, na=False) & (df_full['Category'] == 'FAST FOOD & CONVENIENCE'))]


7-ELEVEN 

In [197]:
df_7_eleven = df_full[df_full['Description'].str.match('7-ELEVEN', case=False, na=False) & (df_full['Category']=='FAST FOOD & CONVENIENCE')]
df_7_eleven.loc[:,'Amount']=df_7_eleven['Amount']+2.0
df_7_eleven_coffee=df_7_eleven.copy()
df_7_eleven_coffee.loc[:,'Amount']=-2.00
df_7_eleven_coffee.loc[:,'Category']='COFFEE'
df_7_eleven_coffee.loc[:,'Master Category']='FOOD AND DINING'

df_7_eleven_transformed = pd.concat([df_7_eleven_coffee, df_7_eleven], ignore_index=True)


In [198]:
df_final=pd.concat([filtered_df, df_7_eleven_transformed], ignore_index=True)

In [199]:
len(df_final)
df_final.head(100)
len(df_final[(df_final['Transaction Date'] >= '2022-06-01') & (df_final['Transaction Date'] <= '2024-07-01')])
sum(df_final.isnull().any(axis=1))


0

In [200]:
len(df_final)

2547

In [201]:
server = 'localhost\\sqlexpress'
database = 'database'

engine = create_engine(f'mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server')

In [202]:
df_final.to_sql('expense_tracking_data', con=engine, if_exists='replace', index=False)


39

In [203]:
# filtered_df.groupby('Standardized Description')['Amount'].sum().sort_values(ascending=True)
# filtered_df['Amount'].sum()
# df_final.to_csv('Transactions Full.csv')

